In [3]:
!pip install -U selenium

     ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
     - -------------------------------------- 0.5/10.0 MB 10.0 MB/s eta 0:00:01
     --- ------------------------------------ 1.0/10.0 MB 10.2 MB/s eta 0:00:01
     ------ --------------------------------- 1.5/10.0 MB 10.8 MB/s eta 0:00:01
     -------- ------------------------------- 2.0/10.0 MB 10.9 MB/s eta 0:00:01
     ---------- ----------------------------- 2.7/10.0 MB 11.5 MB/s eta 0:00:01
     ------------ --------------------------- 3.2/10.0 MB 11.2 MB/s eta 0:00:01
     --------------- ------------------------ 3.9/10.0 MB 11.8 MB/s eta 0:00:01
     ----------------- ---------------------- 4.4/10.0 MB 11.8 MB/s eta 0:00:01
     -------------------- ------------------- 5.2/10.0 MB 12.3 MB/s eta 0:00:01
     ----------------------- ---------------- 5.9/10.0 MB 12.5 MB/s eta 0:00:01
     -------------------------- ------------- 6.6/10.0 MB 12.7 MB/s eta 0:00:01
     ----------------------------- ---------- 7.

In [44]:
!pip install beautifulsoup4

In [154]:
!pip install pytube

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00


In [155]:
cd "C:\Users\USER\Desktop\GitHubCode\CVTeam\Project"

C:\Users\USER\Desktop\GitHubCode\CVTeam\Project


---
# Code

In [172]:
import pandas as pd
import numpy as np
from pytube import YouTube
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt

In [173]:
yt_climb_dataset = pd.DataFrame({'video_Id':[],
                                 'title':[],
                                 'video_url':[],
                                 'sex': [],
                                 'search_term':[],
                                 'start_min':[],
                                 'start_sec':[],
                                 'end_min':[],
                                 'end_sec':[],
                                 'duration':[],
                                 'pose_label':[]})

In [174]:
base_url  = 'http://www.youtube.com'

driver = Chrome()
driver.maximize_window()

driver.get(base_url)
time.sleep(2)
assert 'YouTube' in driver.title

In [175]:
def get_video_info():
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
    
    return soup

def parsing_data():
    soup = get_video_info()
    elem = soup.find_all("ytd-video-renderer", 
                         attrs={"class":'style-scope ytd-item-section-renderer'})
    
    data = []
    for t in elem:
        title = t.find("yt-formatted-string", attrs={"class":'style-scope ytd-video-renderer'}).get_text()
        name = t.find("a", attrs={"class":'yt-simple-endpoint style-scope yt-formatted-string'}).get_text()
        content_url = t.find("a", attrs={"class":'yt-simple-endpoint style-scope ytd-video-renderer'})["href"]
        data.append({'Author': name, 'Title': title, 'URL': 'https://www.youtube.com/' + content_url})
    
    return pd.DataFrame(data)

def scroll_page(page_num= 10):
    body = driver.find_element(By.TAG_NAME,'body')
    while page_num:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        page_num-=1

    return parsing_data()

def search_keyword(keyword):
    search = driver.find_element(By.NAME,'search_query')
    time.sleep(2)
    
    search.send_keys(keyword)
    search.send_keys(Keys.ENTER)
    
    return

In [176]:
search_words=["climbing training",
              "climbing v5",
              "climbing v6",
              "climbing v7",
              "climbing v8",
              "climbing v9",
              "climbing v10",
              "climbing v11",
             ]

In [177]:
dfs = []

for i,element in enumerate(search_words):
    search_keyword(element)
    new_df = scroll_page()
    dfs.append(new_df)
    dfs[i]['search_keyword'] = element
    driver.get(base_url)
    time.sleep(2)
    
test_df = pd.concat(dfs, ignore_index=True)
test_df = test_df.drop_duplicates(subset='Title', keep='first')

In [178]:
test_df['Title'].value_counts()

Title
I Trained Like A Professional Climber for 40 DAYS                                                   1
can i still climb v8 after a YEAR?                                                                  1
How many V8+ boulders can Louis do in 90 minutes??                                                  1
Kept Barndooring on this V8 #bouldering #climb #climbing #movement #heelhook #fortworth #parkour    1
V8! #outdoors #wow #awesome #climb #climbing #tough #foryou #fyp                                    1
                                                                                                   ..
Worlds Hardest V6? ||  1% chance to do this climb!                                                  1
Mental Training for Climbing | How training my EYES helped me send V6                               1
Bouldering v4 to v6: 3 different types of problems                                                  1
V6/7A Boulder Problem #bouldering #shorts #climbing                         

In [179]:
test_df['duration_sec'] = test_df['URL'].map(lambda x: int(YouTube(x).length))

In [180]:
test_df['duration_sec']

0       960
1      1434
2      4149
3      1402
4      1708
       ... 
304      50
305      56
306      86
307      75
308      61
Name: duration_sec, Length: 257, dtype: int64

In [181]:
test_df = test_df.reset_index(drop=True)
test_df

,Author,Title,URL,search_keyword,duration_sec
0,Hannah Morris Bouldering,I Trained Like A Professional Climber for 40 DAYS,https://www.youtube.com//watch?v=txGhHkbPjCk&p...,climbing training,960
1,HoseokClimb,My Rock Climbing Training Routine to V8 Reveal...,https://www.youtube.com//watch?v=IHHEKcfVIsg&p...,climbing training,1434
2,Hooper's Beta,How to Train for Climbing [Full Body Guide ft....,https://www.youtube.com//watch?v=Iy8wi_u5oFI&p...,climbing training,4149
3,Adam Ondra,Climb Smart: Boulder Training Efficiency ⚡ | T...,https://www.youtube.com//watch?v=SLJTwFQE10A&p...,climbing training,1402
4,Adam Ondra,Most Climbers Should Train LIKE THIS | Adam On...,https://www.youtube.com//watch?v=xCu89k1aGWo&p...,climbing training,1708
...,...,...,...,...,...
252,Sam Rothstein,Dreamland V11,https://www.youtube.com//watch?v=G9jW-L8zxAk&p...,climbing v11,50
253,UKClimbing,Highball Fall from Samson (Font 8A/V11) #clim...,https://www.youtube.com//shorts/YnqBwB4EwsU,climbing v11,56
254,Jorge Loredo,“Chung Li” V11 🦧 climbing through a roof!!,https://www.youtube.com//watch?v=gcVg6WEe3J4&p...,climbing v11,86
255,Tension Climbing,"Roy Quanstrom sends ""King Tut"" V11 in Red Feather",https://www.youtube.com//watch?v=kSWIWgTvM-4&p...,climbing v11,75


In [182]:
test_df.to_csv('climb_video_url.csv', index=False)